<a href="https://colab.research.google.com/github/HindJamal97/Algorithm-for-Massive-Data/blob/main/FP_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FIRST PART. 
#Install kaggle and upload + unzip our dataset. 

In [ ]:
!pip install -q kaggle

In [ ]:
#After installing kaggle, we need to upload the kaggle.json file with the API token given from out Kaggle account. 

In [ ]:
from google.colab import files
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hindjamal","key":"5b2794cdd1f8c072feba270d5a765fa6"}'}

In [ ]:
#Now we need to create a Kaggle folder, copy the kaggle.json to the folder created and finally we need to grant an approppriate permission for the json file to act. 

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#To list all the available datasets we have in Kaggle we use the following command

In [ ]:
! kaggle datasets list

ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                                 Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          30036        828  1.0              
tawfikelmetwally/automobile-dataset                                   Car information dataset                               6KB  2023-05-28 18:26:48           2986         83  0.9411765        
fatihb/coffee-quality-data-cqi                                        Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           5314        110  1.0              
mohithsairamreddy/salary-data 

In [ ]:
#Finally, for our aim, we need to upload our dataset, and given that the url is the following one, we can download it as follows: 

In [ ]:
!kaggle datasets download -d  xhlulu/medal-emnlp

100% 6.82G/6.82G [00:55<00:00, 159MB/s]
100% 6.82G/6.82G [00:55<00:00, 133MB/s]


In [ ]:
from zipfile import ZipFile


In [ ]:
file_name = "medal-emnlp.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
#SECOND PART. 
#Install Spark and construct a software that can be used to find frequent itemset (RDD). 

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.3.2
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [ ]:
!pip install py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!java -version

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [ ]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [ ]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b9793a2f43be573e4b73457ca2fcc1ff5250d1fdfc2c8fa81690e62b656b10cc
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext   

In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
sc = spark.sparkContext

In [ ]:
#Upload the file

In [ ]:
dataset_path = '/content/full_data.csv'

In [ ]:
df  = spark.read.format('csv') \
                .option('header', True) \
                .option('multiline', True) \
                .option('quote', '"') \
                .option('escape', '"') \
                .load(dataset_path)
df.printSchema()

root
 |-- TEXT: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LABEL: string (nullable = true)



In [ ]:
df.show()

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
|stroma from eithe...|         6|82|84|107|red cells|serum|a...|
|the effect of the...|                4|13|major|pentose pho...|
|in one experiment...|        32|44|76|135|feeding|feeding|a...|
|the presence of a...|7|15|63|137|199|2...|active|study|acti...|
|the reaction of g...|     113|203|209|250|stable|assay|bind...|
|choline acetyltra...|                  44|             caudate|
|increasing concen...|                  81|        displacement|
|the properties of...|   

In [ ]:
df.head(3)

[Row(TEXT='alphabisabolol has a primary antipeptic action depending on dosage which is not caused by an alteration of the phvalue the proteolytic activity of pepsin is reduced by percent through addition of bisabolol in the ratio of the antipeptic action of bisabolol only occurs in case of direct contact in case of a previous contact with the ATP the inhibiting effect is lost', LOCATION='56', LABEL='substrate'),
 Row(TEXT='a report is given on the recent discovery of outstanding immunological properties in ba ncyanoethyleneurea having a low molecular mass m experiments in ds CS bearing wistar rats have shown that ba at a dosage of only about percent ld mg kg and negligible lethality percent results in a REC rate of percent without hyperglycemia and in one test of percent with hyperglycemia under otherwise unchanged conditions the REF substance ifosfamide if a further development of cyclophosphamide applied without hyperglycemia in its most efficient dosage of percent ld mg kg brought a

In [ ]:
# Get row count
rows = df.count()

# Get columns count
cols = len(df.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")
     

DataFrame Dimensions : (14393619, 3)
DataFrame Rows count : 14393619
DataFrame Columns count : 3


In [ ]:
from pyspark.sql.functions import desc

In [ ]:
df.groupBy("LABEL").count().orderBy(desc("count")).show(10)

+-----------+------+
|      LABEL| count|
+-----------+------+
|      study|294977|
|      after|114472|
|    factors| 71336|
|development| 66812|
|     cancer| 61340|
|      model| 58257|
|     levels| 50882|
|   function| 50024|
|   specific| 44586|
|   approach| 43411|
+-----------+------+
only showing top 10 rows



In [ ]:
Study_Label = df.filter( (df.LABEL == "study"))

In [ ]:
Study_Label.show(10)

+--------------------+--------+-----+
|                TEXT|LOCATION|LABEL|
+--------------------+--------+-----+
|the behaviour of ...|      52|study|
|fast reaction tec...|       6|study|
|the role of the d...|      75|study|
|this study uses a...|      37|study|
|a statistical T0 ...|       2|study|
|the increasing us...|      78|study|
|observation of th...|      33|study|
|this T0 investiga...|       1|study|
|in a T0 of alcoho...|       2|study|
|the authors repor...|      31|study|
+--------------------+--------+-----+
only showing top 10 rows



In [ ]:
Study_Label.groupBy("TEXT").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|                TEXT|count|
+--------------------+-----+
|    retrospective T0|   82|
|  a retrospective T0|   77|
|retrospective coh...|   56|
|   crosssectional T0|   50|
|prospective cohor...|   44|
|      prospective T0|   21|
|a retrospective c...|   15|
|retrospective cli...|   15|
|    a prospective T0|   14|
|      casecontrol T0|   13|
+--------------------+-----+
only showing top 10 rows



In [ ]:
DF = Study_Label.select('TEXT')

In [ ]:
DF.show()

+--------------------+
|                TEXT|
+--------------------+
|the behaviour of ...|
|fast reaction tec...|
|the role of the d...|
|this study uses a...|
|a statistical T0 ...|
|the increasing us...|
|observation of th...|
|this T0 investiga...|
|in a T0 of alcoho...|
|the authors repor...|
|the plantar cushi...|
|in solomon island...|
|during the wet se...|
|su is a unique ne...|
|data are presente...|
|the reconstituted...|
|the cd T0 of the ...|
|the vitamin kdepe...|
|two new methods a...|
|in this introduct...|
+--------------------+
only showing top 20 rows



In [ ]:
#Pre-processing of the data that we have!!

In [ ]:
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
import sparknlp
import math
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline
import pyspark.sql.types as T
from typing import List

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
import time
from tqdm import tqdm

In [ ]:
from pyspark.sql.functions import lower, col, udf

In [ ]:
DF_lowercase = DF.withColumn("processed_text", lower(col("text")))

In [ ]:
DF_lowercase.show()

+--------------------+--------------------+
|                TEXT|      processed_text|
+--------------------+--------------------+
|the behaviour of ...|the behaviour of ...|
|fast reaction tec...|fast reaction tec...|
|the role of the d...|the role of the d...|
|this study uses a...|this study uses a...|
|a statistical T0 ...|a statistical t0 ...|
|the increasing us...|the increasing us...|
|observation of th...|observation of th...|
|this T0 investiga...|this t0 investiga...|
|in a T0 of alcoho...|in a t0 of alcoho...|
|the authors repor...|the authors repor...|
|the plantar cushi...|the plantar cushi...|
|in solomon island...|in solomon island...|
|during the wet se...|during the wet se...|
|su is a unique ne...|su is a unique ne...|
|data are presente...|data are presente...|
|the reconstituted...|the reconstituted...|
|the cd T0 of the ...|the cd t0 of the ...|
|the vitamin kdepe...|the vitamin kdepe...|
|two new methods a...|two new methods a...|
|in this introduct...|in this in

In [ ]:
from pyspark.sql.functions import regexp_replace, trim, col, lower


In [ ]:
def removePunctuation(column):
   return lower(trim(regexp_replace(column,'\p{Punct}',''))).alias('sentence')

In [ ]:
RemoveP = DF_lowercase.select(removePunctuation(col('processed_text')).alias("processed_text"))
RemoveP.show(5)

+--------------------+
|      processed_text|
+--------------------+
|the behaviour of ...|
|fast reaction tec...|
|the role of the d...|
|this study uses a...|
|a statistical t0 ...|
+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import trim,split,explode,col

In [ ]:
tokenizer = Tokenizer(inputCol='processed_text', outputCol='words_token')
Data = RemoveP.withColumn("processed_text", trim(col("processed_text")))
DATA = tokenizer.transform(Data)
DATA.show(20)

+--------------------+--------------------+
|      processed_text|         words_token|
+--------------------+--------------------+
|the behaviour of ...|[the, behaviour, ...|
|fast reaction tec...|[fast, reaction, ...|
|the role of the d...|[the, role, of, t...|
|this study uses a...|[this, study, use...|
|a statistical t0 ...|[a, statistical, ...|
|the increasing us...|[the, increasing,...|
|observation of th...|[observation, of,...|
|this t0 investiga...|[this, t0, invest...|
|in a t0 of alcoho...|[in, a, t0, of, a...|
|the authors repor...|[the, authors, re...|
|the plantar cushi...|[the, plantar, cu...|
|in solomon island...|[in, solomon, isl...|
|during the wet se...|[during, the, wet...|
|su is a unique ne...|[su, is, a, uniqu...|
|data are presente...|[data, are, prese...|
|the reconstituted...|[the, reconstitut...|
|the cd t0 of the ...|[the, cd, t0, of,...|
|the vitamin kdepe...|[the, vitamin, kd...|
|two new methods a...|[two, new, method...|
|in this introduct...|[in, this,

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
Df = remover.transform(DATA).select("processed_text",'words_clean')
Df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#The following code is trying to eliminate some duplicated words that we have in each sentence. 

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType,StringType    

In [ ]:
get_uniques=udf(lambda x: list(set(x)), ArrayType(StringType()))

DataBase = Df.withColumn("words_clean", get_uniques(Df.words_clean))
DataBase.show(10)

+--------------------+--------------------+
|      processed_text|         words_clean|
+--------------------+--------------------+
|the behaviour of ...|[behaviour, human...|
|fast reaction tec...|[protein, means, ...|
|the role of the d...|[production, appe...|
|this study uses a...|[compare, least, ...|
|a statistical t0 ...|[starting, import...|
|the increasing us...|[distribution, le...|
|observation of th...|[behaviour, certa...|
|this t0 investiga...|[appears, new, de...|
|in a t0 of alcoho...|[development, fou...|
|the authors repor...|[certain, second,...|
+--------------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import collect_set
from pyspark.sql.types import IntegerType

In [ ]:
slen = udf(lambda s: len(s), IntegerType())
DB = DataBase.withColumn("word_count", slen(DataBase.words_clean))

In [ ]:
DB.show()

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|the behaviour of ...|[behaviour, human...|        50|
|fast reaction tec...|[protein, means, ...|        61|
|the role of the d...|[production, appe...|        64|
|this study uses a...|[compare, least, ...|        44|
|a statistical t0 ...|[starting, import...|        62|
|the increasing us...|[distribution, le...|        50|
|observation of th...|[behaviour, certa...|        31|
|this t0 investiga...|[appears, new, de...|        55|
|in a t0 of alcoho...|[development, fou...|         8|
|the authors repor...|[certain, second,...|        26|
|the plantar cushi...|[instance, correl...|        51|
|in solomon island...|[additional, prog...|        20|
|during the wet se...|[basis, horses, u...|        59|
|su is a unique ne...|[ill, least, unco...|        52|
|data are presente...|[radiographs, tec...|        50|
|the recon

In [ ]:
import pyspark.sql.functions as F

In [ ]:
DB.createOrReplaceTempView("df_view")

MeDal = spark.sql("SELECT processed_text, words_clean AS items, word_count FROM df_view")

In [ ]:
MeDal.printSchema()

root
 |-- processed_text: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word_count: integer (nullable = true)



In [ ]:
#FPGrowth

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets using support, lift, and confidence.
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.1)
model = fpGrowth.fit(MeDal)

In [ ]:
# Display frequent itemsets.
mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")
mostPopularItemInABasket.show()

+--------------+-----+
|         items| freq|
+--------------+-----+
|         [may]|31004|
|     [may, t0]|31004|
|        [data]|29668|
|    [data, t0]|29668|
|         [aim]|46949|
|     [aim, t0]|46949|
|       [using]|40581|
|   [using, t0]|40581|
|       [study]|42329|
|   [study, t0]|42329|
|        [used]|38166|
|    [used, t0]|38166|
|    [patients]|58487|
|[patients, t0]|58487|
|        [also]|30148|
|    [also, t0]|30148|
|         [two]|30838|
|     [two, t0]|30838|
|     [results]|45547|
| [results, t0]|45547|
+--------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession

In [ ]:
query = """select items, freq
           from mostPopularItemInABasket
           order by freq desc
           limit 20"""
spark.sql(query).show()

+--------------+------+
|         items|  freq|
+--------------+------+
|          [t0]|294977|
|    [patients]| 58487|
|[patients, t0]| 58487|
|         [aim]| 46949|
|     [aim, t0]| 46949|
|     [results]| 45547|
| [results, t0]| 45547|
|       [study]| 42329|
|   [study, t0]| 42329|
|       [using]| 40581|
|   [using, t0]| 40581|
|     [present]| 38754|
| [present, t0]| 38754|
|        [used]| 38166|
|    [used, t0]| 38166|
|         [may]| 31004|
|     [may, t0]| 31004|
|         [two]| 30838|
|     [two, t0]| 30838|
|        [also]| 30148|
+--------------+------+



In [ ]:
query = """select items, freq
           from mostPopularItemInABasket
           where size(items) > 1
           order by freq desc
           limit 20"""
spark.sql(query).show()

+--------------+-----+
|         items| freq|
+--------------+-----+
|[patients, t0]|58487|
|     [aim, t0]|46949|
| [results, t0]|45547|
|   [study, t0]|42329|
|   [using, t0]|40581|
| [present, t0]|38754|
|    [used, t0]|38166|
|     [may, t0]|31004|
|     [two, t0]|30838|
|    [also, t0]|30148|
|    [data, t0]|29668|
+--------------+-----+



In [ ]:
query = """select items, freq
           from mostPopularItemInABasket
           where size(items) > 2
           order by freq desc
           limit 20"""
spark.sql(query).show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [ ]:
# Display generated association rules.
associationRules = model.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show()

+----------+----------+-------------------+----+-------------------+
|antecedent|consequent|         confidence|lift|            support|
+----------+----------+-------------------+----+-------------------+
|   [using]|      [t0]|                1.0| 1.0|0.13757343792905888|
|    [used]|      [t0]|                1.0| 1.0|0.12938635893645944|
|     [may]|      [t0]|                1.0| 1.0|0.10510649982880022|
|     [two]|      [t0]|                1.0| 1.0|0.10454374408852216|
|    [data]|      [t0]|                1.0| 1.0|0.10057733314800815|
|   [study]|      [t0]|                1.0| 1.0|0.14349932367608323|
| [present]|      [t0]|                1.0| 1.0| 0.1313797346911793|
|      [t0]|     [may]|0.10510649982880022| 1.0|0.10510649982880022|
|      [t0]|    [data]|0.10057733314800815| 1.0|0.10057733314800815|
|      [t0]|     [aim]|0.15916156174888213| 1.0|0.15916156174888213|
|      [t0]|   [using]|0.13757343792905888| 1.0|0.13757343792905888|
|      [t0]|   [study]|0.143499323

In [ ]:
associationRules.count()

22

In [ ]:
assoc_rules = associationRules.toPandas()

In [ ]:
assoc_rules.confidence.describe()

count    22.000000
mean      0.566642
std       0.444047
min       0.100577
25%       0.132928
50%       0.599138
75%       1.000000
max       1.000000
Name: confidence, dtype: float64

In [ ]:
assoc_rules.support.describe()

count    22.000000
mean      0.133283
std       0.029499
min       0.100577
25%       0.104684
50%       0.131380
75%       0.151681
max       0.198276
Name: support, dtype: float64

In [ ]:
query = """select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
           from associationRules
           order by confidence desc
           limit 20"""

res = spark.sql(query)
res.show()

+---------------+-----------------+-------------------+
|antecedent (if)|consequent (then)|         confidence|
+---------------+-----------------+-------------------+
|        [using]|             [t0]|                1.0|
|         [used]|             [t0]|                1.0|
|          [may]|             [t0]|                1.0|
|          [two]|             [t0]|                1.0|
|         [data]|             [t0]|                1.0|
|        [study]|             [t0]|                1.0|
|      [present]|             [t0]|                1.0|
|         [also]|             [t0]|                1.0|
|     [patients]|             [t0]|                1.0|
|      [results]|             [t0]|                1.0|
|          [aim]|             [t0]|                1.0|
|           [t0]|       [patients]|0.19827647579302793|
|           [t0]|            [aim]|0.15916156174888213|
|           [t0]|        [results]| 0.1544086488099072|
|           [t0]|          [study]|0.14349932367

In [ ]:
associations = model.transform(MeDal)
associations.show(n=200)

+--------------------+--------------------+----------+--------------------+
|      processed_text|               items|word_count|          prediction|
+--------------------+--------------------+----------+--------------------+
|the behaviour of ...|[behaviour, human...|        50|[may, data, aim, ...|
|fast reaction tec...|[protein, means, ...|        61|[may, data, aim, ...|
|the role of the d...|[production, appe...|        64|[may, data, aim, ...|
|this study uses a...|[compare, least, ...|        44|[may, data, aim, ...|
|a statistical t0 ...|[starting, import...|        62|[may, data, aim, ...|
|the increasing us...|[distribution, le...|        50|[may, data, aim, ...|
|observation of th...|[behaviour, certa...|        31|[may, data, aim, ...|
|this t0 investiga...|[appears, new, de...|        55|[may, data, aim, ...|
|in a t0 of alcoho...|[development, fou...|         8|[may, data, aim, ...|
|the authors repor...|[certain, second,...|        26|[may, data, aim, ...|
|the plantar